# 1 Page Rank 
## a)


In [26]:
import numpy as np
m_a = np.asarray([[0 , 0.5, 0.5],[0, 0, 1],[0, 0, 0]])
display("Matrix A:",m_a)

m_b = np.asarray([[0 , 0.5, 0.5],[0, 0, 1],[0, 0, 1]])

display("Matrix B:",m_b)

'Matrix A:'

array([[0. , 0.5, 0.5],
       [0. , 0. , 1. ],
       [0. , 0. , 0. ]])

'Matrix B:'

array([[0. , 0.5, 0.5],
       [0. , 0. , 1. ],
       [0. , 0. , 1. ]])

In [27]:
def page_rank_naive(matrix, max_step = 10):
    n = len(matrix)
    state = np.asarray([1 / n] * n)
    
    rslt = []
    rslt.append(state)
    for i in range(max_step):
        state = state.dot(matrix)
        rslt.append(state)
    
    return rslt

In [28]:
s = page_rank_naive(m_a)
print("Matrix a")
for i,step in enumerate(s):
    print(i,step)

s = page_rank_naive(m_b)
print("Matrix b")
for i,step in enumerate(s):
    print(i,step)

Matrix a
0 [0.33333333 0.33333333 0.33333333]
1 [0.         0.16666667 0.5       ]
2 [0.         0.         0.16666667]
3 [0. 0. 0.]
4 [0. 0. 0.]
5 [0. 0. 0.]
6 [0. 0. 0.]
7 [0. 0. 0.]
8 [0. 0. 0.]
9 [0. 0. 0.]
10 [0. 0. 0.]
Matrix b
0 [0.33333333 0.33333333 0.33333333]
1 [0.         0.16666667 0.83333333]
2 [0. 0. 1.]
3 [0. 0. 1.]
4 [0. 0. 1.]
5 [0. 0. 1.]
6 [0. 0. 1.]
7 [0. 0. 1.]
8 [0. 0. 1.]
9 [0. 0. 1.]
10 [0. 0. 1.]


-> A leads to all probabilities becoming zero as we have a dead end  
-> B leads to all probability accumulating in the spider trap loop

In [29]:
def has_dead_end(matrix, return_i = False):
    for i in range(len(matrix)):
        if matrix[i].sum() == 0:
            if return_i:
                return True, i
            return True

In [42]:
def page_rank(matrix, max_step = 10, use_beta = True, dead_ends = True, all_results = True):
    beta = 0.85
    n = len(matrix)
    state = np.asarray([1 / n] * n)
    old = state.copy()
    old_matrix = matrix.copy()
    deleted = []
    if dead_ends:
        try:
            while has_dead_end(matrix):
                _, i = has_dead_end(matrix,True)
                deleted.append(i)
                idx = list(set(range(matrix.shape[0])).difference([i]))
                matrix = matrix[np.ix_(idx,idx)]
                sums = matrix.sum(axis=0,keepdims=1); 
                sums[sums==0] = 1
                matrix = matrix/sums
                if matrix.shape == (0,0):
                    raise Exception("Too many Deadends")
        except:
            print("No nodes left when deleting deadends. Aborting..")
            return []
    rslt = []
    rslt.append(state)
    
    for i in range(max_step):
        if use_beta:
            state = beta * state.dot(matrix) + (1-beta) * old
        else:
            state = state.dot(matrix)
        rslt.append(state)
        
    for i in deleted[::-1]:
        #TODO: Implement adding back the stuff
        ranks = rslt[max_step]
        pre_decessors = np.where(old_matrix[:,i] > 0)
        pre_d_power = [1 / len(old_matrix[t].nonzero()) for t in pre_decessors]
        rank = np.sum([s * t for s,t in zip(pre_d_power, ranks[pre_decessors])])
        rslt.append(rslt[:-1].append(rank))
        
    if all_results:
        return rslt
    else:
        return rslt.pop()

In [41]:
print("DeadEnd")
s = page_rank(m_a, use_beta = False)
print("Matrix a")
for i,step in enumerate(s):
    print(i,step)

s = page_rank(m_b, use_beta = False)
print("Matrix b")
for i,step in enumerate(s):
    print(i,step)
    
    
print("Taxation")
s = page_rank(m_a, dead_ends = False)
print("Matrix a")
for i,step in enumerate(s):
    print(i,step)

s = page_rank(m_b, dead_ends = False)
print("Matrix b")
for i,step in enumerate(s):
    print(i,step)
    
print("Both")

s = page_rank(m_a)
print("Matrix a")
for i,step in enumerate(s):
    print(i,step)

s = page_rank(m_b)
print("Matrix b")
for i,step in enumerate(s):
    print(i,step)

DeadEnd
Removed 2
Removed 1
Removed 0
No nodes left when deleting deadends. Aborting..
Matrix a
Matrix b
0 [0.33333333 0.33333333 0.33333333]
1 [0.         0.16666667 0.83333333]
2 [0. 0. 1.]
3 [0. 0. 1.]
4 [0. 0. 1.]
5 [0. 0. 1.]
6 [0. 0. 1.]
7 [0. 0. 1.]
8 [0. 0. 1.]
9 [0. 0. 1.]
10 [0. 0. 1.]
Taxation
Matrix a
0 [0.33333333 0.33333333 0.33333333]
1 [0.05       0.19166667 0.475     ]
2 [0.05       0.07125    0.23416667]
3 [0.05      0.07125   0.1318125]
4 [0.05      0.07125   0.1318125]
5 [0.05      0.07125   0.1318125]
6 [0.05      0.07125   0.1318125]
7 [0.05      0.07125   0.1318125]
8 [0.05      0.07125   0.1318125]
9 [0.05      0.07125   0.1318125]
10 [0.05      0.07125   0.1318125]
Matrix b
0 [0.33333333 0.33333333 0.33333333]
1 [0.05       0.19166667 0.75833333]
2 [0.05    0.07125 0.87875]
3 [0.05    0.07125 0.87875]
4 [0.05    0.07125 0.87875]
5 [0.05    0.07125 0.87875]
6 [0.05    0.07125 0.87875]
7 [0.05    0.07125 0.87875]
8 [0.05    0.07125 0.87875]
9 [0.05    0.07125 0.8

Deadends  
-> A: As there is no node left when removing deadends, we are left without a page rank
-> B: As we do not have dead ends, no change  
Taxation  
-> A: As the surfer disapears with change beta, sum is less than 1
-> B: Works well, spider trap does not accumulate all of the rank anymore
Both
-> A: See above for deadends, same issue occurs
-> B: See above for taxation, there is no change as we do not have deadends

# 2
## a

In [32]:
import pandas as pd
data = pd.read_csv("material/stack_network_links.csv")
transition_matrix = pd.crosstab(data["source"], data["target"])
transition_matrix = transition_matrix.div(transition_matrix.sum(axis=1), axis=0)
display(transition_matrix)

target,.net,agile,ajax,amazon-web-services,android,android-studio,angular,angular2,angularjs,apache,...,visual-studio,vue.js,wcf,web-services,windows,wordpress,wpf,xamarin,xcode,xml
source,,,,,,,,,,,,,,,,,,,,,
.net,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.125000,0.0,0.0,0.0,0.125,0.0,0.0,0.0
agile,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.0
ajax,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.0
amazon-web-services,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.0
android,0.000000,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wordpress,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.0
wpf,0.166667,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.166667,0.0,0.0,0.0,0.000,0.0,0.0,0.0
xamarin,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.0


In [50]:
s = page_rank(transition_matrix.values, all_results=False)
rslt = display(pd.DataFrame(s, index = transition_matrix.index))


,0
source,
.net,0.011388
agile,0.008696
ajax,0.010181
amazon-web-services,0.010420
android,0.007742
...,...
wordpress,0.009520
wpf,0.008687
xamarin,0.002660


# 4
## a

In [ ]:
def support(a,b, all):
    i,c = np.unique(all,return_counts=True)
    counts = dict(zip(i,c))
    return ()